In [1]:
#!/usr/bin/env python
# coding: utf-8

import argparse
import json
from network_utils import get_participants, get_proposals

import pandas as pd
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionContext, ExecutionMode, Executor
from cadCAD import configs

from entities import ProposalStatus
from simulation import (CommonsSimulationConfiguration, bootstrap_simulation,
                        partial_state_update_blocks)
from utils import new_random_number_func


def get_simulation_metrics(df_final):
    average_sentiment = 0


def run_simulation(c: CommonsSimulationConfiguration):
    initial_conditions, simulation_parameters = bootstrap_simulation(c)

    exp = Experiment()
    exp.append_configs(
        initial_state=initial_conditions,
        partial_state_update_blocks=partial_state_update_blocks,
        sim_configs=simulation_parameters
    )

    # Do not use multi_proc, breaks ipdb.set_trace()
    exec_mode = ExecutionMode()
    single_proc_context = ExecutionContext(exec_mode.local_mode)
    executor = Executor(single_proc_context, configs)

    raw_system_events, tensor_field, sessions = executor.execute()

    df = pd.DataFrame(raw_system_events)
    return df


def get_simulation_results(c):
    df = run_simulation(c)
    df_final = df[df.substep.eq(2)]
    random_func = new_random_number_func(None)

    last_network = df_final.iloc[-1, 0]
    candidates = len(get_proposals(last_network, status=ProposalStatus.CANDIDATE))
    actives = len(get_proposals(last_network, status=ProposalStatus.ACTIVE))
    completed = len(get_proposals(last_network, status=ProposalStatus.COMPLETED))
    failed = len(get_proposals(last_network, status=ProposalStatus.FAILED))
    participants = len(get_participants(last_network))

    result = {
        "timestep": list(df_final["timestep"]),
        "funding_pool": list(df_final["funding_pool"]),
        "token_price": list(df_final["token_price"]),
        "sentiment": list(df_final["sentiment"]),
        "score": int(random_func() * 1000),
        "participants": participants,
        "proposals": {
            "candidates": candidates,
            "actives": actives,
            "completed": completed,
            "failed": failed
        }
    }
    return result, df_final




In [2]:
hatchers_count = 35

In [3]:
c = CommonsSimulationConfiguration(timesteps_days=200, days_to_80p_of_max_voting_weight=15, hatchers=hatchers_count, proposals=35, exit_tribute=0.5)
result, df_final = get_simulation_results(c)

COLLATERAL 0.16

                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (200, 9, 1, 8)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 145.83s


In [4]:
df_final

,network,commons,funding_pool,collateral_pool,token_supply,token_price,policy_output,sentiment,simulation,subset,run,substep,timestep
2,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0da7f82f40>,3.267858e+06,1.307143e+07,1.633929e+08,0.160000,None,0.750000,0,0,1,2,1
21,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0da75689d0>,3.268789e+06,1.307711e+07,1.634284e+08,0.160035,"{'failed': [], 'identity': 0, 'succeeded': []}",0.724571,0,0,1,2,2
40,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0da7660a30>,3.269384e+06,1.307966e+07,1.634443e+08,0.160050,"{'failed': [], 'identity': 0, 'succeeded': []}",0.719571,0,0,1,2,3
59,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0da63b89a0>,3.270484e+06,1.308115e+07,1.634536e+08,0.160059,"{'failed': [], 'identity': 0, 'succeeded': []}",0.714571,0,0,1,2,4
78,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0da57923d0>,3.271628e+06,1.308171e+07,1.634571e+08,0.160063,"{'failed': [], 'identity': 0, 'succeeded': []}",0.709571,0,0,1,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3707,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0cfcd8e5b0>,7.806038e+06,9.863910e+05,4.488447e+07,0.043952,"{'failed': [], 'identity': 0, 'succeeded': []}",0.205018,0,0,1,2,196
3726,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0cfbc90fa0>,7.807056e+06,9.859977e+05,4.487552e+07,0.043944,"{'failed': [], 'identity': 0, 'succeeded': []}",0.202431,0,0,1,2,197
3745,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0cfac85250>,7.830046e+06,9.429239e+05,4.388437e+07,0.042973,"{'failed': [], 'identity': 0, 'succeeded': []}",0.199931,0,0,1,2,198
3764,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f0cf99ee640>,7.834695e+06,9.360809e+05,4.372484e+07,0.042817,"{'failed': [], 'identity': 0, 'succeeded': []}",0.197467,0,0,1,2,199


# Metrics

In [5]:
scores = []

## 1 -  price compared to hatch price

In [6]:
hatch_price = df_final.iloc[0, 5]
final_price = df_final.iloc[-1, 5]
ratio_price_1 = final_price/hatch_price
scores.append(ratio_price_1)
print("PRICE RATIO:", ratio_price_1)

PRICE RATIO: 0.2665099856387188


## 2 - Sentiment at the end of the 3 years

In [7]:
final_sentiment_2 = df_final.iloc[-1, 7]
scores.append(final_sentiment_2)
print("FINAL SENTIMENT:", final_sentiment_2)

FINAL SENTIMENT: 0.19509884769007596


## 3 - Number of proposals funded compared to initial proposals

In [8]:
last_network = df_final.iloc[-1, 0]
all_proposals = len(get_proposals(last_network))
candidates = len(get_proposals(last_network, status=ProposalStatus.CANDIDATE))

funded = all_proposals - candidates
initial_proposals = len(get_proposals(df_final.iloc[0, 0], status=ProposalStatus.CANDIDATE))
ratio_funded_3 = funded/initial_proposals
scores.append(ratio_funded_3)
print("RATIO FUNDED BY INITIAL PROPOSALS:", ratio_funded_3)

RATIO FUNDED BY INITIAL PROPOSALS: 0.6


## 4 - Total spent by the funding pool compared to the amount received in the hatch phase

In [9]:
proposals = get_proposals(last_network)

total_spent = 0
for _, proposal in proposals:
    if proposal.status == ProposalStatus.CANDIDATE:
        pass
    else: 
        total_spent += proposal.funds_requested
        
funding_pool_hatch = df_final.iloc[0, 2]

print("FUNDS RECEIVED ON HACTH PHASE:",funding_pool_hatch)     
print("TOTAL SPENT BY THE FUNDING POOL:", total_spent)
ratio_spent_4 = total_spent/funding_pool_hatch
scores.append(ratio_spent_4)
print("RATIO:", ratio_spent_4)

FUNDS RECEIVED ON HACTH PHASE: 3267857.6876969524
TOTAL SPENT BY THE FUNDING POOL: 1776816.9164004244
RATIO: 0.5437253045289894


## 5 - Average amount in funding pool over time compared to the amount received in the hatch phase

In [10]:
avg_funding_pool = df_final["funding_pool"].mean()
funding_pool_hatch = df_final.iloc[0, 2]
avg_funding_pool_ratio_5 = avg_funding_pool/funding_pool_hatch
scores.append(avg_funding_pool_ratio_5)
print("RATIO:", avg_funding_pool_ratio_5)

RATIO: 1.9625242875644755


## 6 - Average price compared with hatch price

In [11]:
avg_price = df_final["token_price"].mean()
hatch_price = df_final.iloc[0, 5]
print("AVG PRICE:", avg_price, "HATCH PRICE:", hatch_price)
avg_price_ratio_6 = avg_price/hatch_price
scores.append(avg_price_ratio_6)
print("PRICE RATIO:", avg_price_ratio_6)

AVG PRICE: 0.09690906779524049 HATCH PRICE: 0.16
PRICE RATIO: 0.605681673720253


## 7 - Average sentiment

In [12]:
avg_sentiment_7 = df_final["sentiment"].mean()
scores.append(avg_sentiment_7)
print("AVERAGE SENTIMENT:", avg_sentiment_7)

AVERAGE SENTIMENT: 0.40255283436722555


## 8 - Ratio of successful projects to failed ones

In [13]:
completed = len(get_proposals(last_network, status=ProposalStatus.COMPLETED))
failed = len(get_proposals(last_network, status=ProposalStatus.FAILED))

print("COMPLETED:", completed, "FAILED:", failed)
ratio_completed_failed_8 = completed/failed
scores.append(ratio_completed_failed_8)
print("RATIO completed/failed", ratio_completed_failed_8)

COMPLETED: 6 FAILED: 14
RATIO completed/failed 0.42857142857142855


## 9 - Nº of final participants compared to Nº of hatchers

In [14]:
last_network = df_final.iloc[-1, 0]

participants = len(get_participants(last_network))
print("PARTICIPANTS:",participants)
ratio_participants_9 = participants/hatchers_count
scores.append(ratio_participants_9)
print("RATIO hatchers/partcicipants", ratio_participants_9)

PARTICIPANTS: 39
RATIO hatchers/partcicipants 1.1142857142857143


# Final Score

In [15]:
print(scores)
print("FINAL SCORE", sum(scores)*100)

[0.2665099856387188, 0.19509884769007596, 0.6, 0.5437253045289894, 1.9625242875644755, 0.605681673720253, 0.40255283436722555, 0.42857142857142855, 1.1142857142857143]
FINAL SCORE 611.8950076366881
